# Basic MusicXML to MIDI converter/parser

***

## Uses codebase from https://github.com/mac-marg-pianist/musicXML_parser

***

# Install Dependencies

In [ ]:
%cd /content
!git clone https://github.com/asigalov61/tegridy-tools

# Import Modules

In [ ]:
import os
import tqdm
import random

if not os.path.exists('/content/XMLs'):
    os.makedirs('/content/XMLs')

if not os.path.exists('/content/MIDIs'):
    os.makedirs('/content/MIDIs')

%cd /content/tegridy-tools/tegridy-tools/
import TMIDIX

%cd /content/tegridy-tools/tegridy-tools/musicxml_parser
from main import *

%cd /content

# Convert

In [ ]:
sorted_or_random_file_loading_order = True # Sorted order is NOT usually recommended
dataset_ratio = 1 # Change this if you need more data


print('XML to MIDI Processor')
print('Starting up...')

###########

files_count = 0

gfiles = []

melody_chords_f = []

###########

print('Loading XML files...')
print('This may take a while on a large dataset in particular.')

dataset_addr = "/content/XMLs"

# os.chdir(dataset_addr)
filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]
print('=' * 70)

if filez == []:
    print('Could not find any XML files. Please check XMLs dir...')
    print('=' * 70)

if sorted_or_random_file_loading_order:
    print('Sorting files...')
    filez.sort()
    print('Done!')
    print('=' * 70)
else:
    print('Randomizing file list...')
    random.shuffle(filez)


print('Processing XML files. Please wait...')
for f in tqdm.tqdm(filez[:int(len(filez) * dataset_ratio)]):

        dir = os.path.dirname(f).split('/')[-1]

        fn = os.path.basename(f)
        fn1 = fn.split('.')[0]

        try:
          XML = MusicXMLDocument(f)

          MIDI_resolution = XML.midi_resolution

          key_signature = XML.get_key_signatures()[0].__str__()

          notes, rests = XML.get_notes()

          song = []

          time = 0
          for n in notes:
            patch = n.midi_program
            
            time = n.note_duration.time_position * 550
            dur = n.note_duration.midi_ticks
            chan = n.midi_channel
            pitch = n.pitch[1]
            velocity = n.velocity

            song.append(['note', time, dur, chan, pitch, velocity])

          fname = '/content/MIDIs/'+ fn1

          TMIDIX.Tegridy_SONG_to_MIDI_Converter(song, 
                                                output_signature=fn1,
                                                track_name=key_signature,
                                                number_of_ticks_per_quarter=500, 
                                                output_file_name=fname)
          #break

        except KeyboardInterrupt:
          break
        
        except:
          print('Bad XML:', f)
          continue

# Congrats! You did it! :)